# This file begins pulling senate contributions into a dataframe

In [4]:
import requests,json,pprint
import pandas as pd
import pprint

url = "https://lda.senate.gov/api/v1/contributions"
response = requests.get(url).json()
pprint.pprint(response)

{'count': 535040,
 'next': 'https://lda.senate.gov/api/v1/contributions/?page=2',
 'previous': None,
 'results': [{'address_1': '1817 Edgehill Drive',
              'address_2': None,
              'city': 'Alexandria',
              'comments': None,
              'contact_name': None,
              'contribution_items': [{'amount': '500.00',
                                      'contribution_type': 'feca',
                                      'contribution_type_display': 'FECA',
                                      'contributor_name': 'FULLER, CAROLYN',
                                      'date': '2008-03-09',
                                      'honoree_name': 'Senator Daniel Inouye',
                                      'payee_name': 'DAN10'},
                                     {'amount': '500.00',
                                      'contribution_type': 'feca',
                                      'contribution_type_display': 'FECA',
                                  

In [7]:
# This section does a couple things:
# 1. Begins populating new_dict_list with information we want. Later it is converted to a df
# 2. Counts how many results exist and how many contributions exist in each result to make sense 
# of what is being pulled versus what is present. Essentially, to make sure all are accounted for.
# 3. It only loops through the first three pages as a test

new_dict_list = []

# Even though the URL is already defined above, if you don't define it again here,
# it will start at the last page left off as it keeps updating the global variable with the last page.
url = "https://lda.senate.gov/api/v1/contributions"

# Reset cummulativeContributions
cummulativeContributions = 0

# Loop through first three pages
for i in range(3):
    print(url)
    response = requests.get(url).json()
    resultCount = 0
    totalContributions = 0
    for eachResult in response["results"]:
        contributionCount = 0
        for eachContribution in eachResult["contribution_items"]:
            new_entry = {"amount":eachContribution["amount"],
                         "contributor_name":eachContribution["contributor_name"],
                         "date":eachContribution["date"],
                         "honoree_name":eachContribution["honoree_name"]}      
            new_dict_list.append(new_entry)
            contributionCount =contributionCount+1
        print(f"Contribution count is {contributionCount}")
        resultCount =resultCount+1
        totalContributions = totalContributions + contributionCount
    print(f"Result count is {resultCount}")
    print(f"Total Contributions for this page is {totalContributions}")
    cummulativeContributions = cummulativeContributions + totalContributions
    url = response["next"]
        
    
print(f"Cummulative contributions is {cummulativeContributions}")
df = pd.DataFrame(new_dict_list)

df.shape

https://lda.senate.gov/api/v1/contributions
Contribution count is 6
Contribution count is 0
Contribution count is 0
Contribution count is 0
Contribution count is 10
Contribution count is 18
Contribution count is 12
Contribution count is 0
Contribution count is 0
Contribution count is 0
Contribution count is 1
Contribution count is 1
Contribution count is 0
Contribution count is 0
Contribution count is 0
Contribution count is 1
Contribution count is 2
Contribution count is 0
Contribution count is 0
Contribution count is 0
Contribution count is 8
Contribution count is 0
Contribution count is 0
Contribution count is 1
Contribution count is 0
Result count is 25
Total Contributions for this page is 60
https://lda.senate.gov/api/v1/contributions/?page=2
Contribution count is 0
Contribution count is 0
Contribution count is 0
Contribution count is 1
Contribution count is 0
Contribution count is 1
Contribution count is 0
Contribution count is 0
Contribution count is 0
Contribution count is 0
Co

(92, 4)

In [3]:
df

,amount,contributor_name,date,honoree_name
0,500.00,"FULLER, CAROLYN",2008-03-09,Senator Daniel Inouye
1,500.00,"FULLER, CAROLYN",2008-06-26,Congressman Mike Simpson
2,250.00,"FULLER, CAROLYN",2008-03-09,Congressman Bill Sali
3,2500.00,"FULLER, CAROLYN",2008-04-01,Senator Jon Tester
4,500.00,"FULLER, CAROLYN",2008-06-25,Senator Max Baucus
...,...,...,...,...
87,500.00,"LINK, JIM",2008-04-17,Senator Mike Enzi
88,1000.00,"LINK, JIM",2008-06-20,Congressman David Davis
89,500.00,"LINK, JIM",2008-03-11,Congressman Lincoln Diaz-Balart
90,500.00,"LINK, JIM",2008-03-11,Congressman Mario Diaz-Balart
